# Hyrax Dynamic Aggregation function via NcML

The NcML module can be used to aggregate multiple datasets into one virtual dataset.
Hyrax currently support three of the NcML aggregations:
* union
* joinNew
* joinExisiting

In this example, we leverage the *joinNew Aggregation* which joins existing datasets along a new outer Array dimension. Essentially, it adds a new index to the existing variable which points into the values in each member dataset. One useful example of this aggregation is for joining multiple samples of data from different times into one virtual dataset containing all the times.

Reference https://docs.opendap.org/index.php?title=Dynamic_Aggregation_Tutorial

In [1]:
from pydap.client import open_url
import requests

## Sea Ice Concentration products aggregation example

### Single product

#### OpenDAP

In [2]:
url = 'https://hyrax.terradue.com/opendap/data/amsr2/n6250/netcdf/2019/asi-AMSR2-n6250-20190101-v5.4.nc'
r = requests.get(url + '.dds')
print(r.text)

Dataset {
    String polar_stereographic;
    Float64 x[x = 1216];
    Float64 y[y = 1792];
    Grid {
      Array:
        Float32 z[y = 1792][x = 1216];
      Maps:
        Float64 y[y = 1792];
        Float64 x[x = 1216];
    } z;
} asi-AMSR2-n6250-20190101-v5.4.nc;



### Time series

#### NcML

```
<?xml version="1.0" encoding="UTF-8"?>
<!-- This test and data is taken from the aggregation tutorial page at
http://www.unidata.ucar.edu/software/thredds/current/netcdf-java/ncml/Aggregation.html -->
<netcdf title="joinNew aggregation using the scan element and dateFormatMark">

  <aggregation type="joinNew" dimName="time">
    <variableAgg name="x"/>
    <variableAgg name="y"/>
    <variableAgg name="z"/>
    <scan location="data/amsr2/n6250/netcdf"
          suffix=".nc"
          subdirs="true"
          regExp="^.*/asi-AMSR2-n6250-[^/]*"
          dateFormatMark="asi-AMSR2-n6250-#yyyyMMdd"
          />
  </aggregation>

<!-- example: asi-AMSR2-n6250-20120702-v5.4.nc -->

</netcdf>
```

#### OpenDAP

In [3]:
url = 'https://hyrax.terradue.com/opendap/data/ts/amsr2.ncml'
r = requests.get(url + '.dds')
print(r.text)

Dataset {
    String polar_stereographic;
    Float64 x[time = 2478][x = 1216];
    Float64 y[time = 2478][y = 1792];
    String time[time = 2478];
    Grid {
      Array:
        Float32 z[time = 2478][y = 1792][x = 1216];
      Maps:
        String time[time = 2478];
        Float64 y[y = 1792];
        Float64 x[x = 1216];
    } z;
} amsr2.ncml;



In [5]:
sic = open_url(url)
keys = list(sic.keys())
for key in keys:
    print(key)

polar_stereographic
x
time
y
z


In [7]:
sic.time[0:10].data

array(['2012-07-02T00:00:00Z', '2012-07-03T00:00:00Z',
       '2012-07-04T00:00:00Z', '2012-07-05T00:00:00Z',
       '2012-07-06T00:00:00Z', '2012-07-07T00:00:00Z',
       '2012-07-08T00:00:00Z', '2012-07-09T00:00:00Z',
       '2012-07-10T00:00:00Z', '2012-07-11T00:00:00Z'], dtype='<U20')

In [9]:
sic.z[0]

<GridType with array 'z' and maps 'time', 'y', 'x'>

## Sea Ice Thickness products aggregation example

### Single product

#### OpenDAP

In [10]:
url = 'https://hyrax.terradue.com/opendap/data/smos_smap/netCDF/north/2018/20180419_north_mix_sit_v100.nc'
r = requests.get(url + '.dds')
print(r.text)

Dataset {
    Float32 smos_thickness[X = 896][Y = 608];
    Float32 smos_thickness_unc[X = 896][Y = 608];
    Float32 smap_thickness[X = 896][Y = 608];
    Float32 smap_thickness_unc[X = 896][Y = 608];
    Float32 combined_thickness[X = 896][Y = 608];
    Float32 combined_thickness_unc[X = 896][Y = 608];
    Int16 flags[X = 896][Y = 608];
} 20180419_north_mix_sit_v100.nc;



### Time series

#### NcML

```
<?xml version="1.0" encoding="UTF-8"?>
<!-- This test and data is taken from the aggregation tutorial page at
http://www.unidata.ucar.edu/software/thredds/current/netcdf-java/ncml/Aggregation.html -->
<netcdf title="joinNew aggregation using the scan element and dateFormatMark">

  <aggregation type="joinNew" dimName="time">
    <variableAgg name="smos_thickness"/>
    <variableAgg name="smos_thickness_unc"/>
    <variableAgg name="smap_thickness"/>
    <variableAgg name="smap_thickness_unc"/>
    <variableAgg name="combined_thickness"/>
    <variableAgg name="combined_thickness_unc"/>
    <scan location="data/smos_smap/netCDF"
          suffix=".nc"
          subdirs="true"
          regExp="^.*/.*_mix_sit_v100[^/]*"
          dateFormatMark="#yyyyMMdd_"
          />
  </aggregation>

<!-- example: 20151231_north_mix_sit_v100.nc-->

</netcdf>
```

#### OpenDAP

In [11]:
url = 'https://hyrax.terradue.com/opendap/data/ts/smos_smap.ncml'
r = requests.get(url + '.dds')
print(r.text)

Dataset {
    Int16 flags[X = 896][Y = 608];
    Float32 smos_thickness[time = 504][X = 896][Y = 608];
    Float32 smos_thickness_unc[time = 504][X = 896][Y = 608];
    String time[time = 504];
    Float32 smap_thickness[time = 504][X = 896][Y = 608];
    Float32 smap_thickness_unc[time = 504][X = 896][Y = 608];
    Float32 combined_thickness[time = 504][X = 896][Y = 608];
    Float32 combined_thickness_unc[time = 504][X = 896][Y = 608];
} smos_smap.ncml;



In [12]:
sit = open_url(url)
keys = list(sit.keys())
for key in keys:
    print(key)

flags
smos_thickness
smos_thickness_unc
smap_thickness
smap_thickness_unc
time
combined_thickness
combined_thickness_unc


In [13]:
sit.smos_thickness.shape

(504, 896, 608)

In [15]:
sit.time[0:10].data

array(['2015-03-31T00:00:00Z', '2015-04-01T00:00:00Z',
       '2015-04-02T00:00:00Z', '2015-04-03T00:00:00Z',
       '2015-04-04T00:00:00Z', '2015-04-05T00:00:00Z',
       '2015-04-06T00:00:00Z', '2015-04-07T00:00:00Z',
       '2015-04-08T00:00:00Z', '2015-04-09T00:00:00Z'], dtype='<U20')

In [16]:
sit.smos_thickness[0]

<BaseType with data array([[[-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
          1.12241678e-01,  8.07810202e-02,  8.07810202e-02],
        [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
          1.16029546e-01,  7.45694861e-02,  7.45694861e-02],
        [-9.99000000e+02, -9.99000000e+02, -9.99000000e+02, ...,
          1.16029546e-01,  1.17853865e-01,  4.73127738e-02],
        ...,
        [ 2.16191605e-01,  2.62407094e-01,  2.62407094e-01, ...,
         -9.99000000e+02, -9.99000000e+02, -9.99000000e+02],
        [ 2.16191605e-01,  1.71574935e-01,  1.71574935e-01, ...,
         -9.99000000e+02, -9.99000000e+02, -9.99000000e+02],
        [-9.99000000e+02,  1.71574935e-01,  1.71574935e-01, ...,
         -9.99000000e+02, -9.99000000e+02, -9.99000000e+02]]],
      dtype=float32)>